## How gwlearn Differs from MGWR

`gwlearn` is a **modern, flexible alternative** to traditional geographically weighted regression libraries like [`mgwr`](https://github.com/pysal/mgwr). While both enable spatial regression analysis, they have fundamentally different design philosophies.

## mgwr architecture:
![Current MGWR](https://raw.githubusercontent.com/FirePheonix/mgwrVSgwlearnSVGs/eb291a4a19a6fd6fb0dc1f09a20e9d0c78c36d0d/mgwr_architecture.svg)


## gwlearn architecture::

![GWLearn Architecture](https://raw.githubusercontent.com/FirePheonix/mgwrVSgwlearnSVGs/eb291a4a19a6fd6fb0dc1f09a20e9d0c78c36d0d/gwlearn_architecture.svg)

## Setup: Load Common Data

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import time
from geodatasets import get_path
from sklearn import metrics

# Load dataset
gdf = gpd.read_file(get_path("geoda.guerry"))
X = gdf[['Crm_prp', 'Litercy', 'Donatns', 'Lottery']]
y = gdf['Suicids']
geom = gdf.representative_point()
coords = np.column_stack([geom.x, geom.y])  # For mgwr


## Both can do: Linear GWR

In [5]:
# NOTE: run the same cell twice:
# The first gwlearn run includes joblib worker initialization and numerical backend warm-up
# subsequent runs reuse these resources, resulting in significantly lower execution time.

# gwlearn
from gwlearn.linear_model import GWLinearRegression

start = time.time()
gw = GWLinearRegression(geometry=geom, bandwidth=25, fixed=False)
gw.fit(X, y)
gw_time = time.time() - start
print(f"gwlearn R²: {metrics.r2_score(y, gw.pred_):.4f}  AICc: {gw.aicc_:.1f}  Time: {gw_time:.2f}s")

# mgwr
from mgwr.gwr import GWR

start = time.time()
mg = GWR(coords, y.values.reshape(-1,1), X.values, bw=25, fixed=False).fit()
mg_time = time.time() - start
print(f"mgwr R²: {metrics.r2_score(y, mg.predy):.4f}  AICc: {mg.aicc:.1f}  Time: {mg_time:.2f}s")

print(f"\n  mgwr is {gw_time/mg_time:.1f}x faster for basic linear GWR")

gwlearn R²: 0.7251  AICc: 2007.4  Time: 0.46s
mgwr R²: 0.7251  AICc: 2011.2  Time: 0.08s

  mgwr is 5.7x faster for basic linear GWR


### Why the Timing Difference?


**mgwr approach:**
- Builds a single block-diagonal weight matrix
- Solves weighted least squares using optimized numpy/scipy linear algebra
- Computes all local coefficients in ONE vectorized operation
- Minimal Python overhead - mostly C-level numpy operations

![mgwr_time](https://raw.githubusercontent.com/FirePheonix/mgwrVSgwlearnSVGs/89a10b18ddcd06c88d9108f3f1fc0e4054e641bd/mgwr_time_difference.svg)

**gwlearn approach:**
- Creates a libpysal.graph.Graph for spatial weights
- For each location, clones the sklearn estimator
- Dispatches N parallel jobs via joblib 
    - THIS is the exact step is time expensive because:
         Python overhead N times
         Each local model:

            1.  Calls fit()

            2. Validates inputs

            3. Handles sample weights

            4. Runs sklearn plumbing

        Even if each fit is “small”, Python call overhead accumulates. Even thogh each model solves

        $\hat{\beta} = (X^{\mathsf T} W_i X)^{-1} X^{\mathsf T} W_i y$

        it become expensive since gwlearn repeats the fitting setup N times.
 
        

        
- Each job fits an independent sklearn model with sample weights
- Aggregates results from all local models

![gwlearn_time](https://raw.githubusercontent.com/FirePheonix/mgwrVSgwlearnSVGs/0c9f9bca761d016f24af24fb91de4da2eb571869/gwlearn_time_difference.svg)

**mgwr is faster for basic linear GWR** because:

| Factor | mgwr | gwlearn |
|--------|------|---------|
| **Implementation** | Specialized matrix solver | Generic meta-estimator |
| **Computation** | Single vectorized WLS | N separate sklearn model fits |
| **Overhead** | Minimal (direct numpy) | joblib + sklearn cloning |
| **Optimization** | Hand-tuned for GWR | Flexibility over speed |

**Key insight**: mgwr solves GWR as ONE matrix equation across all locations simultaneously using optimized linear algebra. gwlearn fits N independent sklearn models (one per location), which adds overhead but enables **any sklearn estimator**.

**Trade-off**:
-  **Need speed + only linear GWR?** → Use mgwr
-  **Need flexibility (Ridge, Lasso, RF, classification)?** → Use gwlearn

---
##  1. Architecture Comparison: API Style

**MGWR**: Custom API with coordinate arrays  
**gwlearn**: sklearn-compatible API with GeoSeries

In [6]:
from mgwr.gwr import GWR

# mgwr requires: numpy arrays, coords as separate array, y reshaped
mgwr_model = GWR(
    coords=coords,                    # Coordinate array (n, 2)
    y=y.values.reshape(-1, 1),        # Must reshape to (n, 1)
    X=X.values,                       # Feature array
    bw=25,
    fixed=False,
    kernel='bisquare'
)
mgwr_results = mgwr_model.fit()

print("MGWR API:")
print(f"- Input: numpy arrays + coordinate matrix")
print(f"- Output: Custom results object")
print(f"- R²: {metrics.r2_score(y, mgwr_results.predy):.4f}")

MGWR API:
- Input: numpy arrays + coordinate matrix
- Output: Custom results object
- R²: 0.7251


In [7]:
from gwlearn.linear_model import GWLinearRegression

# gwlearn uses: pandas DataFrames, GeoSeries for geometry
gwlearn_model = GWLinearRegression(
    geometry=geom,                    # GeoSeries (integrated)
    bandwidth=25,
    fixed=False,
    kernel='bisquare'
)
gwlearn_model.fit(X, y)               # Standard sklearn .fit(X, y)

print("gwlearn API:")
print(f"- Input: pandas DataFrame + GeoSeries")
print(f"- Output: Fitted estimator with attributes")
print(f"- R²: {metrics.r2_score(y, gwlearn_model.pred_):.4f}")

gwlearn API:
- Input: pandas DataFrame + GeoSeries
- Output: Fitted estimator with attributes
- R²: 0.7251


---
## 2. Model Fitting: Same Results, Different Approach

In [8]:
from numpy.testing import assert_array_almost_equal, assert_almost_equal

print("VALIDATION: gwlearn matches MGWR output")

# Predictions
assert_array_almost_equal(gwlearn_model.pred_, mgwr_results.predy.flatten())
print("Predictions: MATCH")

# Local R²
assert_array_almost_equal(gwlearn_model.local_r2_, mgwr_results.localR2.flatten())
print("Local R²: MATCH")

# Coefficients
assert_array_almost_equal(gwlearn_model.local_intercept_, mgwr_results.params[:, 0])
assert_array_almost_equal(gwlearn_model.local_coef_, mgwr_results.params[:, 1:])
print("Coefficients: MATCH")

# Information criteria
print(f"\nAIC: gwlearn={gwlearn_model.aic_:.2f}  |  mgwr={mgwr_results.aic:.2f}")
print(f"BIC: gwlearn={gwlearn_model.bic_:.2f}  |  mgwr={mgwr_results.bic:.2f}")
print(f"AICc: gwlearn={gwlearn_model.aicc_:.2f} |  mgwr={mgwr_results.aicc:.2f}")

VALIDATION: gwlearn matches MGWR output
Predictions: MATCH
Local R²: MATCH
Coefficients: MATCH

AIC: gwlearn=1960.75  |  mgwr=1960.75
BIC: gwlearn=2045.63  |  mgwr=2045.63
AICc: gwlearn=2007.43 |  mgwr=2011.20


## Reason for difference in AICc values:

The corrected Akaike Information Criterion (AICc) is defined as:

$\mathrm{AICc} = \mathrm{AIC} + \frac{2k(k+1)}{n - k - 1}$


where:
- \( n \) is the number of observations  
- \( k \) is the effective number of model parameters  

**Key point:**  
Although `gwlearn` and `mgwr` produce *identical predictions, coefficients, and likelihood-based metrics (AIC, BIC)*, their AICc values may differ slightly.

This discrepancy arises from **different definitions of the effective degrees of freedom (\(k\))**:
- `mgwr` estimates \(k\) using **hat-matrix trace–based measures**, which are more conservative.
- `gwlearn` uses a **simpler estimator-aligned parameter count**, resulting in a slightly smaller \(k\).

As a result, AICc values can differ by a small margin even when the fitted models are numerically identical. This difference reflects **model complexity accounting**, not a difference in model fit.


---
## 3. Model Support Comparison

**MGWR**: Only linear regression  
**gwlearn**: Any sklearn estimator

In [9]:
# gwlearn: GW-Ridge (impossible in MGWR)

from gwlearn.base import BaseRegressor
from sklearn.linear_model import Ridge

gw_ridge = BaseRegressor(
    model=Ridge,
    geometry=geom,
    bandwidth=25,
    fixed=False,
    alpha=1.0  # Regularization parameter passed to Ridge
)
gw_ridge.fit(X, y)

print(f"GW-Ridge R²: {metrics.r2_score(y, gw_ridge.pred_):.4f}")

GW-Ridge R²: 0.1235


In [10]:
# gwlearn: GW-Lasso (impossible in MGWR)

from sklearn.linear_model import Lasso

gw_lasso = BaseRegressor(
    model=Lasso,
    geometry=geom,
    bandwidth=25,
    fixed=False,
    alpha=0.1
)
gw_lasso.fit(X, y)

print(f"GW-Lasso R²: {metrics.r2_score(y, gw_lasso.pred_):.4f}")

GW-Lasso R²: 0.1230


In [11]:
# gwlearn: GW-RandomForest Classification (impossible in MGWR)

from gwlearn.ensemble import GWRandomForestClassifier

# Create binary target for classification
y_binary = (y > y.median()).astype(int)

gw_rf = GWRandomForestClassifier(
    geometry=geom,
    bandwidth=40,
    fixed=False,
    n_estimators=50,
    random_state=42
)
gw_rf.fit(X, y_binary)

mask = ~gw_rf.pred_.isna()
print(f"GW-RandomForest Accuracy: {metrics.accuracy_score(y_binary[mask], gw_rf.pred_[mask]):.4f}")
print(f"Models fitted: {mask.sum()}/{len(mask)}")

GW-RandomForest Accuracy: 0.6941
Models fitted: 85/85


In [12]:
# gwlearn: GW-GradientBoosting Classification (impossible in MGWR)

from gwlearn.ensemble import GWGradientBoostingClassifier

gw_gb = GWGradientBoostingClassifier(
    geometry=geom,
    bandwidth=40,
    fixed=False,
    n_estimators=50,
    random_state=42
)
gw_gb.fit(X, y_binary)

mask = ~gw_gb.pred_.isna()
print(f"GW-GradientBoosting Accuracy: {metrics.accuracy_score(y_binary[mask], gw_gb.pred_[mask]):.4f}")
print(f"Models fitted: {mask.sum()}/{len(mask)}")
print("(Completely impossible in MGWR)")

GW-GradientBoosting Accuracy: 0.6588
Models fitted: 85/85
(Completely impossible in MGWR)


---
## 4. Kernel Weighting Comparison

Both support similar kernels, but gwlearn integrates with libpysal.graph

In [13]:
# Available kernels in gwlearn
kernels = ['triangular', 'parabolic', 'bisquare', 'tricube', 'cosine', 'boxcar']

print("Kernel comparison across libraries:")
results = []

for kernel in kernels:
    gw = GWLinearRegression(geometry=geom, bandwidth=25, fixed=False, kernel=kernel)
    gw.fit(X, y)
    r2 = metrics.r2_score(y, gw.pred_)
    results.append({'Kernel': kernel, 'R²': r2, 'AICc': gw.aicc_})

pd.DataFrame(results)

Kernel comparison across libraries:


,Kernel,R²,AICc
0,triangular,0.723878,2003.382362
1,parabolic,0.630305,1993.697495
2,bisquare,0.725147,2007.433836
3,tricube,0.703052,2005.984826
4,cosine,0.648737,1995.075608
5,boxcar,0.507931,1981.012221


---
## 5. Bandwidth Search Comparison

In [14]:
# MGWR: Bandwidth selection (Sel_BW)

from mgwr.sel_bw import Sel_BW

start = time.time()
mgwr_selector = Sel_BW(coords, y.values.reshape(-1, 1), X.values, fixed=False, kernel='bisquare')
mgwr_optimal_bw = mgwr_selector.search(criterion='AICc')
mgwr_time = time.time() - start

print(f"MGWR Bandwidth Search:")
print(f"Optimal bandwidth: {mgwr_optimal_bw}")
print(f"Time: {mgwr_time:.2f}s")
print(f"Criterion: AICc only")

MGWR Bandwidth Search:
Optimal bandwidth: 70.0
Time: 0.57s
Criterion: AICc only


In [16]:
# gwlearn: BandwidthSearch (more flexible)

from gwlearn.search import BandwidthSearch

start = time.time()
gwlearn_search = BandwidthSearch(
    GWLinearRegression,
    geometry=geom,
    fixed=False,
    kernel='bisquare',
    search_method='golden_section',  # or 'interval'
    criterion='aicc',                 # Can also use: 'aic', 'bic', 'log_loss', custom
    min_bandwidth=10,
    max_bandwidth=80,
)
gwlearn_search.fit(X, y)
gwlearn_time = time.time() - start

print(f"\ngwlearn Bandwidth Search:")
print(f"Optimal bandwidth: {gwlearn_search.optimal_bandwidth_}")
print(f"Time: {gwlearn_time:.2f}s")
print(f"Criterion: Flexible (aicc, aic, bic, log_loss, custom)")
print(f"Methods: golden_section, interval")


gwlearn Bandwidth Search:
Optimal bandwidth: 69
Time: 3.93s
Criterion: Flexible (aicc, aic, bic, log_loss, custom)
Methods: golden_section, interval


In [17]:
# gwlearn: Interval search with multiple metrics
interval_search = BandwidthSearch(
    GWLinearRegression,
    geometry=geom,
    fixed=False,
    search_method='interval',
    criterion='aicc',
    metrics=['aic', 'bic'],  # Track additional metrics
    min_bandwidth=15,
    max_bandwidth=60,
    interval=5,
)
interval_search.fit(X, y)

print("\nInterval Search Results:")
print(interval_search.scores_)
print(f"\nOptimal bandwidth: {interval_search.optimal_bandwidth_}")


Interval Search Results:
15    2112.050494
20    2032.033644
25    2007.433836
30    1991.763417
35    1983.540290
40    1978.314198
45    1974.192606
50    1971.369979
55    1970.657519
60    1969.622931
Name: aicc, dtype: float64

Optimal bandwidth: 60


##  6. Parallelization Comparison
(run this cell more than once to see consistent, accurate timing results without warm up time.)

In [19]:
# gwlearn with n_jobs=1 (single-threaded)
start = time.time()
gw_single = GWLinearRegression(geometry=geom, bandwidth=25, fixed=False, n_jobs=1)
gw_single.fit(X, y)
single_time = time.time() - start

# gwlearn with n_jobs=-1 (all cores)
start = time.time()
gw_parallel = GWLinearRegression(geometry=geom, bandwidth=25, fixed=False, n_jobs=-1)
gw_parallel.fit(X, y)
parallel_time = time.time() - start

print(f"gwlearn Parallelization:")
print(f"Single-threaded (n_jobs=1): {single_time:.3f}s")
print(f"Parallel (n_jobs=-1):       {parallel_time:.3f}s")
print(f"Speedup: {single_time/parallel_time:.1f}x")
print(f"\n Note: MGWR has limited parallelization support")

gwlearn Parallelization:
Single-threaded (n_jobs=1): 0.923s
Parallel (n_jobs=-1):       0.430s
Speedup: 2.1x

 Note: MGWR has limited parallelization support
